In [16]:
import os, sklearn, pandas, numpy as np, collections
from sklearn import svm
import skimage, skimage.io, skimage.filters
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# custom scripts
import config # params, constants
import data, models # functions that mutate outr data
from utils import utils, plot # custom functions, in local environment

Using TensorFlow backend.


 Dataset :: namedtuple(
    ['train' = ['img_name']
    , 'test' = ['img_name']
    , 'labels' = pandas.df('img_name','breed')
    , 'dict_index_to_label' = dict to convert label_index -> label_name
    , 'dict_label_to_index'= dict to convert label_name -> label_index
    


In [3]:
import data # src/data.py
dataset = data.init_dataset()

In [4]:
# nr of instances per class
counter = collections.Counter(dataset.labels['breed'])
list(counter.items())[0:4]

[('boston_bull', 87), ('dingo', 80), ('pekinese', 75), ('bluetick', 85)]

In [5]:
# amount of classes
amt = len(counter.items())
amt

120

In [6]:
# sort classes by nr of instances
ls = list(counter.items())
ls.sort(key=lambda x: x[1], reverse=True)
# data = {k:v for k,v in ls[:] }
ls[:3]

[('scottish_deerhound', 126), ('maltese_dog', 117), ('afghan_hound', 116)]

In [7]:
top_classes = [label for label,value in ls[:3]]
top_classes

['scottish_deerhound', 'maltese_dog', 'afghan_hound']

In [8]:
dataset.labels.loc[dataset.labels['breed'] == top_classes[0]]

,id,breed
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
161,03dc61595ad9dbf49e3998cf586ca8cb,scottish_deerhound
233,056b535b441278e83839984f1b1da0a6,scottish_deerhound
235,05719b998e57a11b863a322ecc7652a5,scottish_deerhound
237,058d4bfce39c9f1157ca4c1b019bbf1a,scottish_deerhound
362,08946697b3e50e602d6bea765c8fb9f5,scottish_deerhound
480,0b72856996ad5826afff7195ff678589,scottish_deerhound
489,0bad79c06832a1bdeaedbbb50a28450a,scottish_deerhound
599,0eaea4c6fe055607ed8322db0306285f,scottish_deerhound


In [9]:
data_per_class = [ dataset.labels.loc[dataset.labels['breed'] == top_classes[i]] for i in range(len(top_classes))]
len(data_per_class)

3

In [10]:
data_per_class[0]

,id,breed
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
161,03dc61595ad9dbf49e3998cf586ca8cb,scottish_deerhound
233,056b535b441278e83839984f1b1da0a6,scottish_deerhound
235,05719b998e57a11b863a322ecc7652a5,scottish_deerhound
237,058d4bfce39c9f1157ca4c1b019bbf1a,scottish_deerhound
362,08946697b3e50e602d6bea765c8fb9f5,scottish_deerhound
480,0b72856996ad5826afff7195ff678589,scottish_deerhound
489,0bad79c06832a1bdeaedbbb50a28450a,scottish_deerhound
599,0eaea4c6fe055607ed8322db0306285f,scottish_deerhound


In [12]:
data.top_classes(dataset.labels)

[('scottish_deerhound', 126), ('maltese_dog', 117), ('afghan_hound', 116)]

In [15]:
data.items_with_label(dataset.labels, 'scottish_deerhound')

,id,breed
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
161,03dc61595ad9dbf49e3998cf586ca8cb,scottish_deerhound
233,056b535b441278e83839984f1b1da0a6,scottish_deerhound
235,05719b998e57a11b863a322ecc7652a5,scottish_deerhound
237,058d4bfce39c9f1157ca4c1b019bbf1a,scottish_deerhound
362,08946697b3e50e602d6bea765c8fb9f5,scottish_deerhound
480,0b72856996ad5826afff7195ff678589,scottish_deerhound
489,0bad79c06832a1bdeaedbbb50a28450a,scottish_deerhound
599,0eaea4c6fe055607ed8322db0306285f,scottish_deerhound
